In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from collections import Counter
from sklearn.pipeline import make_pipeline
import sklearn as sk
import sys
import joblib

In [2]:
inputs = pd.read_csv('inputs.txt', sep=' ')
labels = pd.read_csv('labels.txt', sep=' ')

data = pd.DataFrame(np.c_[inputs.values, labels.values])
data.head()



,0,1,2,3,4,5,6,7,8,9,...,2343,2344,2345,2346,2347,2348,2349,2350,2351,2352
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0


In [3]:
data.describe()

,0,1,2,3,4,5,6,7,8,9,...,2343,2344,2345,2346,2347,2348,2349,2350,2351,2352
count,1999.0,1999.0,1999.0,1999.0,1999.0,1999.0,1999.0,1999.0,1999.0,1999.0,...,1999.0,1999.0,1999.0,1999.0,1999.0,1999.0,1999.0,1999.0,1999.0,1999.000000
mean,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.495748
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.850260
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
25%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.000000
50%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.000000
75%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.000000
max,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.000000


In [4]:

training_set, validation = train_test_split(data, test_size = 0.2, random_state = 21)
X_train = training_set.iloc[:,0:-1].values
y_train = training_set.iloc[:,-1].values
X_test = validation.iloc[:,0:-1].values

y_test = validation.iloc[:,-1].values

In [5]:
print("train size X : ",X_train.shape)
print("train size y : ",y_train.shape)
print("train size X : ",X_test.shape)
print("train size y : ",y_test.shape)

train size X :  (1599, 2352)
train size y :  (1599,)
train size X :  (400, 2352)
train size y :  (400,)


In [6]:

print(Counter(y_train))
print(Counter(y_test))

Counter({1.0: 186, 7.0: 183, 9.0: 175, 4.0: 167, 2.0: 165, 6.0: 154, 3.0: 154, 8.0: 146, 5.0: 143, 0.0: 126})
Counter({1.0: 50, 4.0: 47, 6.0: 46, 7.0: 45, 0.0: 44, 3.0: 38, 2.0: 37, 5.0: 36, 8.0: 32, 9.0: 25})


In [7]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold

model = LogisticRegression(multi_class='multinomial', solver='lbfgs',  max_iter=1000)

# define the model evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate the model and collect the scores
n_scores = cross_val_score(model, X_test, y_test, scoring='accuracy', cv=cv, n_jobs=-1)
# report the model performance
print('Mean Accuracy: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

Mean Accuracy: 0.813 (0.059)


In [8]:
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000, multi_class='multinomial')

In [9]:
yhat = model.predict(X_test)

print('Predicted Class: %d' % yhat[0])

print(yhat[399], y_test[399])

Predicted Class: 4
4.0 4.0


In [10]:
yhat = model.predict_proba(X_test)
# summarize the predicted probabilities
print('Predicted Probabilities: %s' % yhat[0])

Predicted Probabilities: [6.18900681e-05 2.70973785e-06 1.29450232e-05 1.14278724e-07
 9.94915588e-01 3.79910308e-04 1.84901810e-05 3.57879733e-03
 5.06936641e-04 5.22618503e-04]


In [11]:
filename = 'model.sav'
joblib.dump(model, filename)
print("done")

done


In [12]:
np.set_printoptions(threshold=sys.maxsize)

#print(labels)

In [13]:
input = np.loadtxt('inputs.txt')
